# 모듈 다운로드

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import plotly.express as px

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.font_manager as fm

In [ ]:
!pip install plotly

In [ ]:
!pip install folium

In [ ]:
from folium.plugins import MarkerCluster

In [ ]:
!pip install pyproj

In [ ]:
import pyproj

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 결측값 확인

In [ ]:
data_origin = pd.read_csv("C:/Users/cbskust/Dropbox/교과/다차원자료분석PBL/기말고사/데이터/data.csv")

In [ ]:
pd.DataFrame(data_origin.isna().sum()).T.style.background_gradient(cmap='rainbow')

# 이상치 제거

In [ ]:
# PRICE 열의 IQR 계산
Q1 = data_origin['PRICE'].quantile(0.25)
Q3 = data_origin['PRICE'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# IQR 기준으로 이상치 제거
data = data_origin[(data_origin['PRICE'] >= lower_bound) & (data_origin['PRICE'] <= upper_bound)]

In [ ]:
box_original = px.box(data_origin, y='PRICE', title='Box Plot of Original Data',
                          labels={'PRICE': 'PRICE'})

# 이상치 제거 후 박스 플롯 그리기
box_no_outliers = px.box(data, y='PRICE', title='Box Plot without outliers',
                             labels={'PRICE': 'PRICE'})


box_original.write_image("1.png", scale = 3)
box_no_outliers.write_image("2.png", scale = 3)
# 그래프 출력
box_original.show()
box_no_outliers.show()

## 데이터 엑셀과 csv로 저장

In [ ]:
data.to_csv("C:/Users/cbskust/Dropbox/교과/다차원자료분석PBL/기말고사/데이터/data_outlier_eliminated_csv.csv", index = False, encoding = 'utf-8-sig')

In [ ]:
data.to_excel('C:/Users/cbskust/Dropbox/교과/다차원자료분석PBL/기말고사/데이터/data_outlier_eliminated_excel.xlsx', index = False, engine = 'openpyxl')

# 기초통계량 확인

In [ ]:
pd.DataFrame(data[['PRICE', 'TOTAL_POP', 'TOTAL_WORKER', 'BUILDING', 'BUS_N', 'MART_DEPARTMENT', 'WASTE_EFFECT']].describe()).T.style.format("{:.2f}")

# 데이터 시각화

In [ ]:
data

In [ ]:
data.info()

In [ ]:
plt.rcParams['font.family']

In [ ]:
font_path = 'C:/Windows/Fonts/malgun.ttf'

In [ ]:
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='SIGUNGU_NAME', data=data, color = 'grey')
plt.title('Counter plot of SIGUNGU_NAME')
plt.xlabel('SIGUNGU_NAME', fontsize = 12)
plt.ylabel('Count')
plt.xticks(fontsize = 7)
plt.rc('font', family = font_name)
plt.show()

## 상관행렬

In [ ]:
selected_columns = ['PRICE', 'TOTAL_POP', 'TOTAL_WORKER', 'BUILDING', 'BUS_N', 'MART_DEPARTMENT', 'WASTE_EFFECT']  
selected_data = data[selected_columns]

# 상관 행렬 계산
corr_matrix = selected_data.corr()

# 상관 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Selected Variables')
plt.show()

In [ ]:
# 특정 변수 선택
x_var = 'TOTAL_WORKER'  # x축에 사용할 변수명
y_var = 'PRICE'   # y축에 사용할 변수명

# 산점도 그리기
fig = px.scatter(data, x=x_var, y=y_var, title=f'Scatter Plot of {x_var} vs {y_var}', labels={x_var: x_var, y_var: y_var})

fig.update_layout(
    width=700,  # 그래프의 너비
    height=700  # 그래프의 높이
)

fig.show()

In [ ]:
# 특정 변수 선택
x_var = 'BUILDING'  # x축에 사용할 변수명
y_var = 'TOTAL_POP'   # y축에 사용할 변수명

# 산점도 그리기
fig = px.scatter(data, x=x_var, y=y_var, title=f'Scatter Plot of {x_var} vs {y_var}', labels={x_var: x_var, y_var: y_var})

fig.update_layout(
    width=700,  # 그래프의 너비
    height=700  # 그래프의 높이
)

fig.show()

In [ ]:
selected_columns = data.columns

In [ ]:
selected_columns

In [ ]:
selected_data = data[selected_columns]

# VIF 계산을 위해 상수항 추가
X = selected_data.drop(columns=['PRICE', 'PNU', 'DONG_CODE', 'SIGUNGU_NAME', 'DONG_NAME', 'BON', 'BU', 'X', 'Y',
       'LAND_USE_CODE'])  # 종속 변수 제거
X = sm.add_constant(X)  # 상수항 추가

# VIF 계산
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 결과 출력
print(vif_data)

# 변수 간 관계 분석